In [1]:
import os
import io
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pykrx import stock
from pandas_datareader import data as pdr
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas_datareader'

In [ ]:
# --- 초기 변수 설정 ---
today = datetime.today()
crawl_time = datetime.now()
start_week = (today - timedelta(weeks=200)).strftime("%Y%m%d")
start_day  = (today - timedelta(days=200)).strftime("%Y%m%d")
end        = today.strftime("%Y%m%d")

maPeriod      = 30
flatThresh    = 0.0005
gapThresh     = 0.03
valueThr      = 500_000_000   # 주간 거래대금 최소 기준
spikeMult     = 1.2           # 변동성 배수 기준
minDailyVol   = 300_000       # 일간 거래량 최소 기준

In [2]:
# 전략 상태 계산 함수
def calc_stage(ma, price, slope):
    flat = abs(slope / ma) < flatThresh
    gap  = abs(price - ma) / ma < gapThresh
    if flat:
        return 1 if gap else 3
    return (2 if price > ma else 1) if slope > 0 else (4 if price < ma else 3)

In [3]:
# 개별 종목 스캔 함수
def scan_ticker(args):
    ticker, name_map, sector_map, cap_map = args
    try:
        # 일간 OHLCV 조회 (pykrx)
        df = stock.get_market_ohlcv_by_date(start_week, end, ticker, freq='d')
        if df.empty or len(df) < 80:
            return None
        # 거래대금 컬럼
        if '거래대금' not in df.columns:
            df['거래대금'] = df['종가'] * df['거래량']
        df.index = pd.to_datetime(df.index)

        # 주간 리샘플링
        wk = df.resample('W-FRI').agg({
            '시가':'first','고가':'max','저가':'min','종가':'last',
            '거래량':'sum','거래대금':'sum'
        }).dropna()
        if len(wk) < 16:
            return None

        # 주간 지표 계산
        wk['ma']         = wk['종가'].rolling(maPeriod).mean()
        wk['slope']      = wk['ma'].diff()
        wk['turnover']   = wk['종가'] * wk['거래량']
        wk['ma30']       = wk['종가'].rolling(30).mean()
        wk['ma30_slope'] = wk['ma30'].diff()
        latest, prev = wk.iloc[-1], wk.iloc[-2]
        stage = calc_stage(latest['ma'], latest['종가'], latest['slope'])
        stage_prev = calc_stage(prev['ma'], prev['종가'], prev['slope'])

        # 완화된 주간 조건
        week_ok = latest['turnover'] >= valueThr and latest['종가'] > latest['ma30']

        # 일간 데이터
        dk = stock.get_market_ohlcv_by_date(start_day, end, ticker, freq='d')
        if dk.empty:
            return None
        dk['vol_ma20'] = dk['거래량'].rolling(20).mean()
        latest_d, prev_d = dk.iloc[-1], dk.iloc[-2]

        # 완화된 일간 조건
        day_ok = latest_d['종가'] > latest_d['시가'] and latest_d['거래량'] >= minDailyVol

        if stage == 2 and stage_prev == 1 and week_ok and day_ok:
            return {
                'Ticker': ticker,
                'Name':   name_map.get(ticker),
                'Sector': sector_map.get(ticker, "Unknown"),
                'MarketCap': cap_map.get(ticker, 0),
                'Stage': stage,
                'Entry_Price': float(latest['종가']),
                'Last_Close': float(latest_d['종가']),
                'Last_Close_Date': f"{dk.index[-1].strftime('%Y-%m-%d')} {crawl_time.strftime('%H:%M:%S')}"
            }
    except Exception:
        return None
    return None

In [4]:
# 메인 블록: 티커 리스트 로드 및 매핑 (pandas_datareader 대체 및 pykrx 순수-Python 대안)
kospi  = stock.get_market_ticker_list(market="KOSPI")
kosdaq = stock.get_market_ticker_list(market="KOSDAQ")
tickers = kospi + kosdaq

# Name mapping
name_map = {t: stock.get_market_ticker_name(t) for t in tickers}

# Sector mapping (현재 Unknown으로 설정)
sector_map = {t: "Unknown" for t in tickers}

# MarketCap mapping (pykrx를 이용한 시가총액 조회)
cap_df = stock.get_market_cap_by_date(end, end)
cap_series = cap_df.loc[end]
cap_map = cap_series.to_dict()

args_list = [(t, name_map, sector_map, cap_map) for t in tickers]

NameError: name 'end' is not defined

In [5]:
# 스캔 실행: 병렬 처리
with Pool(cpu_count()) as pool:
    results = list(tqdm(pool.imap(scan_ticker, args_list), total=len(args_list)))
candidates = [r for r in results if r]

if not candidates:
    print("No candidates found.")
else:
    print(f"Found {len(candidates)} candidates.")

NameError: name 'Pool' is not defined

In [6]:
# 결과 처리 및 표시
df = pd.DataFrame(candidates)
df['MarketCap(억원)'] = df['MarketCap'] / 1e8
df = df[['Ticker', 'Name', 'Sector', 'MarketCap(억원)', 'Stage', 'Entry_Price', 'Last_Close', 'Last_Close_Date']]
df = df.sort_values(['Stage', 'MarketCap(억원)'], ascending=[True, False])
df.head()

NameError: name 'candidates' is not defined

In [7]:
# 결과 저장: Excel 생성 및 차트 추가
output_path = "krx_weinstein_scan_updated.xlsx"
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    df.to_excel(writer, index=False, sheet_name="Scan")
    scan_ws = writer.sheets["Scan"]
    for idx, col in enumerate(df.columns):
        length = df[col].astype(str).map(len).max()
        scan_ws.set_column(idx, idx, max(length, len(col)) + 2)
    detail_ws = writer.book.add_worksheet("Detail")
    start_3m = (today - timedelta(days=90)).strftime("%Y%m%d")
    for i, ticker in enumerate(df['Ticker']):
        hist = stock.get_market_ohlcv_by_date(start_3m, end, ticker, freq='d')
        if hist.empty:
            continue
        fig, ax = plt.subplots()
        ax.plot(pd.to_datetime(hist.index), hist['종가'], label='Close')
        entry = df.loc[df['Ticker'] == ticker, 'Entry_Price'].values[0]
        ax.axhline(entry, linestyle='--', label='Entry Price')
        name = df.loc[df['Ticker'] == ticker, 'Name'].values[0]
        ax.set_title(f"{name} 3개월 일봉 차트")
        ax.legend()
        fig.tight_layout()
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        plt.close(fig)
        buf.seek(0)
        detail_ws.insert_image(i * 20, 0, f"{ticker}.png", {'image_data': buf, 'x_scale': 0.5, 'y_scale': 0.5})
print(f"Scan complete: {df.shape[0]} candidates found. Results saved to {output_path}")

NameError: name 'df' is not defined